In [36]:
# Nothing to see here! Just run it at the start of the presentation
from util import invmod
from math import log
from random import randint

def invmod(e, n):
    for d in range(n):
        if d * e % n == 1:
            return d

primes = [2]
testPrime = 3
while testPrime < 10000:
    isprime = True
    for prime in primes:
        if testPrime % prime == 0:
            isprime = False
            break
    if isprime:
        primes.append(testPrime)
    testPrime += 2
    
primes = set(primes)
print('done')

done


# The Mathematics of RSA Encryption
### The Africa Congruence Part II

FieldStack Lunch & Learn

July 2019

Nathan Dunn

# Goals
* Make encryption less mysterious
* Learn some cool math

<center>
    <a href="https://youtu.be/g8uR3zhxujU">
        <img style="height:315px" src="images/africa-congruence.png">
    </a>
</center>

We may know conceptually what encryption is doing, but not really what it's doing under the hood

Much of the math will be similar to the Africa Congruence, but seeing it isn't a prerequisite

# Outline
* Asymmetric Encryption
* How do you perform RSA encryption?
* Where do keys come from?
* Why does it work?
* Why is it secure?

Sprinkled throughout we'll have plenty of mathemtical tangents

# What is RSA Encryption?
* Asymmetric Encryption Scheme
    * Other implementations are possible

# What is Asymmetric Encryption?
<center>
    <img style="height:600px" src="images/asymmetric-encryption.png">
</center>

Contrary to symmetric encryption where you have one key to encrypt data and the same key can be used to decrypt the data, in asymmetric encryption, there are 2 keys.

<center>
    <img style="height:700px" src="images/encrypting.png">
</center>

<center>
    <img style="height:700px" src="images/signing.png">
</center>

<center>
    <img style="height:700px;" src="images/both.png">
</center>

That sounds pretty good, but how do we do it?
First, we need some tools

# Math Tools - Messages as Numbers

* Say you want to encode the message `Hi!`
* Each character has an ASCII code
    * `H` $\rightarrow 72_{10}  \rightarrow 01001000_2$
    * `i` $\rightarrow 105_{10} \rightarrow 01101001_2$
    * `!` $\rightarrow 33_{10}  \rightarrow 00100001_2$
* Concatenate the bits so you have one big number
    * $010010000110100100100001_2$
    * $474550510_{10}$

# Math Tools - Modulus



In [6]:
546 % 31

19

* “The Remainder” operator

<center>
    <img style="height:300px" src="images/long-division.png">
</center>

* `546 % 31 == 19`

* $546 = 19 \mod 31$

* $546 = k \cdot 31 + 19$
    * $k = 17$

So for instance, if you have a song that's 31 seconds long, and you've been listening to it on repeat for 546 seconds, you are 19 seconds into the 18th playthrough

Turns out that's all the math you need to know to encrypt and decrypt a message

# The RSA Algorithm
* Public Key: $(e,n)$
* Private Key: $(d, n)$
* Message: $M$


* $EM = M^e \mod n$
* $DM = EM^d \mod n$

# RSA Example

In [13]:
# Public key
(e, n) = (3593, 150349)
# Private key - Don't tell!
(d, n) = (958,  150349)

# Message
M = 90002

In [14]:
# Encrypted Message
EM = M**e % n
print(EM)

25366


In [15]:
# Decrypted Message
DM = EM**d % n
print(DM)

91516


Hopefully the decrypted message is the same as the original message!

Normally two different parties would have done the encryption and decryption

Do an example with a different message M - completely different cyphertext, even though the cleartext is very similar

Show that changing the key will screw it up

Show that using that encrypting using the *private* exponent still works

Clearly there's something special about these keys

# Where do the keys come from?


# Math Tools - Primes
* A **prime number** can only be divided by $1$ and itself
    * In contrast to a **composite number**
* Every number can be **factored** into a list of primes
    * $360 = 2 \cdot 2 \cdot 2 \cdot 3 \cdot 3 \cdot 5$
    * $11 = 11$
* Two numbers are **coprime** if they have no common prime factors
    * $6 = 2 \cdot 3, \space \space 35 = 5 \cdot 7$, so $6$ and $35$ are coprime
    * $26 = \underline{2} \cdot 13, \space\space 4 = \underline{2} \cdot 2 \space$, so $26$ and $4$ are not coprime


# Math Tools - Totient Function
* $\phi(n)$ is called the **totient** of $n$
    * Number of integers less than $n$, coprime with $n$
* $\phi(p \cdot q) = (p-1) \cdot (q-1)$

# Key Generation
1. Choose random primes, $p$ and $q$
1. Calculate modulus $n = p \cdot q$
1. Choose an arbitrary prime value for $e$
1. Calculate $\phi(n) = (p-1) \cdot (q-1)$
1. Solve $d \cdot e = 1 \mod \phi(n)$

In [16]:
# Choose random primes
p, q = 251, 599
# Calculate modulus as product of primes
n = p * q
print(n)

150349


In [17]:
# φ(n)
phi_n = (p-1) * (q-1)
print(phi_n)

149500


In [35]:
# Arbitrarily choose e
e = 3593

# Solve d*e % φ(n) = 1
d = invmod(e, phi_n)
print(d)

957


Could use the Chinese Remainder Theorem to solve for d. But let's black box the algorithm

In [19]:
# Verify
print(d * e % phi_n)

1


In [20]:
# Final result
print('Public key  ',  (e, n))
print('Private key ', (d, n))
 

Public key   (3593, 150349)
Private key  (957, 150349)


# What makes those keys work?

What is it about the key generation process that makes them able to encrypt and decrypt a message

# Math Tools - Euler's Theorem

$$\text{$x$ is coprime with $n$} \implies x^{\phi(n)} = 1 \mod n$$

$$EM = M^e \mod n$$
$$DM = EM^d \mod n$$
$$DM = M^{e \cdot d} \mod n$$

$$e \cdot d = 1 \mod \phi(n) \implies e \cdot d = 1 + k \cdot \phi(n)$$


$$M^{e \cdot d} = M^{1 + k \cdot \phi(n)} = M^1 \cdot (M ^ {\phi(n)})^k$$

$$M^{e \cdot d} \mod n = M \cdot (1) ^ k \mod n$$

$$M ^ {e \cdot d} \mod n = M$$

# Why is it secure?

All we've done so far is show that we can do some mathematical transformation and then reverse it. But the same could be said for adding 7 and then subtracting 7, so clearly we need to do more work to show why it's secure

## Can an attacker get the private key from the public key?
* $e \cdot d = 1 \mod \phi(n)$
* $3593 \cdot d = 1 \mod \phi(150349)$

* $3593 \cdot d = 1 \mod 149500$
    * Can an attacker calculate $\phi(n) = (p-1) \cdot (q-1)$

We have all this information that we had before

* No, because **factoring** is difficult

## RSA-200
* 663 bit number from the RSA Factoring Challenge

* Factored in 2005

* 2 years real time, 75 years CPU time

In [21]:
rsa200 = 27997833911221327870829467638722601621070446786955428537560009929326128400107609345671052955360856061822351910951365788637105954482006576775098580557613579098734950144178863178946295187237869221823983
len(str(rsa200))

200

In [22]:
p = 3532461934402770121272604978198464368671197400197625023649303468776121253679423200058547956528088349
q = 7925869954478333033347085841480059687737975857364219960734330341455767872818152135381409304740185467

rsa200 == p*q

True

## RSA-2048
<span style="overflow-wrap: break-word;">25195908475657893494027183240048398571429282126204032027777137836043662020707595556264018525880784406918290641249515082189298559149176184502808489120072844992687392807287776735971418347270261896375014971824691165077613379859095700097330459748808428401797429100642458691817195118746121515172654632282216869987549182422433637259085141865462043576798423387184774447920739934236584823824281198163815010674810451660377306056201619676256133844143603833904414952634432190114657544454178424020924616515723350778707749817125772467962926386356373289912154831438167899885040445364023527381951378636564391212010397122822120720357</span>

* Prize of $200,000
* This is the recommended size of modern RSA keys

## Can an attacker solve for $d$ given a signed message?
* $EM = M^d \mod n$
* $6646 = 90001 ^ d \mod 150349$

Instead of trying to get the key given the private key, what if we have some extra information. Namely, the cleartext and cyphertext of a given message.

* No, because **discrete logarithms** are difficult

## Can an attacker solve for $M$ given the encrypted message?
* $EM = M^e \mod n$
* $131435 = M ^ {3593} \mod 150349$

What if we want to settle for just breaking a single message, rather than breaking the private key

* No, because taking the $e^{th}$ root is difficult
    * Known as the RSA problem

# Questions?

# Appendix
* Proof that $\phi(p \cdot q) = (p-1) \cdot (q-1)$
* Proof of Euler's Theorem
* Generating Prime Numbers

# Argument that $\phi(p \cdot q) = (p-1) \cdot (q-1)$

* $\phi(n)$ is called the **totient** of $n$
    * Number of integers less than $n$, coprime with $n$
* $n = 15 = 5 \cdot 3$
* $\phi(15) =$ Number of integers coprime with $15$

In [23]:
for i in range(15):
    if i % 3 == 0 and i % 5 == 0:
        print('FizzBuzz')
    elif i % 3 == 0:
        print('Fizz')
    elif i % 5 == 0:
        print('Buzz')
    else:
        print(i)    

FizzBuzz
1
2
Fizz
4
Buzz
Fizz
7
8
Fizz
Buzz
11
Fizz
13
14


$\phi(15) = 8$

$\phi(p \cdot q) = p \cdot q - p - q + 1$

$\phi(p \cdot q) = (p-1) \cdot (q-1)$

$\phi(5 \cdot 3) = (5-1) \cdot (3-1) = 8$

# Argument for Euler's Theorem
$$\text{$x$ is coprime with $n$} \implies x^{\phi(n)} = 1 \mod n$$

Euler's Theorem predicts

$$\text{$x$ is coprime with $15$} \implies x^8 = 1 \mod 15$$

In [24]:
for i in range(15):
    print(i, i**8 % 15)

0 0
1 1
2 1
3 6
4 1
5 10
6 6
7 1
8 1
9 6
10 10
11 1
12 6
13 1
14 1


## $7^8 \mod 15$

$1 \cdot 2 \cdot 4 \cdot 7 \cdot 8 \cdot 11 \cdot 13 \cdot 14 \mod 15$

$7^8 (1 \cdot 2 \cdot 4 \cdot 7 \cdot 8 \cdot 11 \cdot 13 \cdot 14) \mod 15$

$(7 \cdot 1) (7 \cdot 2) (7 \cdot 4) (7 \cdot 8) (7 \cdot 11) (7 \cdot 13) (7 \cdot 14) \mod 15$

<center>
    <img style="height:600px;" src="images/multiplication-table.png">
</center>

<center>
    <img style="height:600px;" src="images/multiplication-table-highlighted.png">
</center>

## $7^8 \mod 15$
$1 \cdot 2 \cdot 4 \cdot 7 \cdot 8 \cdot 11 \cdot 13 \cdot 14 \mod 15$

$7^8 (1 \cdot 2 \cdot 4 \cdot 7 \cdot 8 \cdot 11 \cdot 13 \cdot 14) \mod 15$

$(7 \cdot 1) (7 \cdot 2) (7 \cdot 4) (7 \cdot 8) (7 \cdot 11) (7 \cdot 13) (7 \cdot 14) \mod 15$

$7 \cdot 14 \cdot 13 \cdot 4 \cdot 11 \cdot 2 \cdot 1 \cdot 8 \mod 15$

$\therefore 7^8 = 1 \mod 15$

Consider the product of all the coprime values, not that we care about the actual value

Multiply this value by 7^8

Multiplying each number by 7 just rearranges the values, so the product doesn't change!

# Generating Prime Numbers

1. Generate a random number of the correct size
1. Test if that number if prime
1. If yes, done. If not, go to 1.

* How many numbers will we have to test?
* Isn't testing primality difficult?

If we have to test trillions of numbers to find a prime, kind of a non-starter...how many do we need to test?

If we have to factor the value to figure out if it's prime, that's also a problem

# Distribution of Primes
$$\pi(N) \sim \frac{N}{\log(N)} \implies \text{One out of $\log(N)$ numbers of size $N$ are prime}$$


In [33]:
for N in [10, 100, 1000, 2**1024]:
    print(log(N))

2.302585092994046
4.605170185988092
6.907755278982137
709.782712893384


Accuracy of this approximation improves with greater N

To generate a 1024-bit prime, we have to check around 710 values on average...no biggie

# Primality Testing

* Fermat Primality Test
    * Probabilistic
* Fermat's Little Theorem
    * $p \text{ prime} \implies x^{p-1} = 1 \mod p$

Turns out, you can determine *if* a number has prime factors without actually finding the factors!

Probabilistic - small chance of falsely reporting a number is prime, but can be made arbitrarily small

Fermat's Little Theorem is a special case of Euler's Theorem

* Do **k** times
    * Pick **x** randomly in the range [2,N-2]
    * If $x^{N-1} \ne 1 \mod N$, $N$ is composite
    * If Fermat's expression holds all $k$ times, $N$ is likely prime

In [31]:
def isPrime(N):
    k = 2
    for _ in range(k):
        a = randint(2, N-2)
        remainder = a**(N-1) % N
        if remainder != 1:
            return False
    return True

In [32]:
incorrect = 0
for val in range(5, 10000):
    if isPrime(val) != (val in primes):
        incorrect += 1
        
print(incorrect)

1


Try increasing k to 10

# Sources
* https://en.wikipedia.org/wiki/RSA_(cryptosystem)